In [1]:
%cd ../..
%run cryptolytic/notebooks/init.ipynb
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import cryptolytic.util as util
import cryptolytic.start as start
import cryptolytic.viz.plot as plot
import cryptolytic.data.sql as sql
import cryptolytic.data as d
from cryptolytic.util import *
import cryptolytic.data.historical as h
import cryptolytic.model as m

from matplotlib.pylab import rcParams
from IPython.core.display import HTML
from pandas.plotting import register_matplotlib_converters # to stop a warning message


ohclv = ['open', 'high', 'close', 'low', 'volume']
plt.style.use('ggplot')
rcParams['figure.figsize'] = 20,7
start.init()
register_matplotlib_converters()


# Make math readable
HTML("""
<style>
.MathJax {
    font-size: 2rem;
}
</style>""")

/home/me/Documents/Git/Lambda-School-Labs/cryptolytic-ds
Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
df = d.get_df ({'start':'06-01-2019', 'period':300, 'trading_pair':'eth_btc', 'exchange_id':'bitfinex'}, n=15000)
# df2 = d.get_df({'start':'05-07-2019', 'period':300, 'trading_pair':'btc_usd', 'exchange_id':'hitbtc'}, n=15000)
# df3 = d.get_df({'start':'05-07-2019', 'period':300, 'trading_pair':'btc_usdt', 'exchange_id':'poloniex'}, n=15000)
# df2 = d.get_df({'start':'04-05-2019', 'period':300, 'trading_pair':'eth_btc', 'exchange_id':'hitbtc'}, n=2000)

/home/me/Documents/Git/Lambda-School-Labs/cryptolytic-ds/cryptolytic/data/__init__.py:54: FutureWarning:

how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)



In [3]:
# TODO upgrade tensorflow to 2
# import tensorflow_transform as tft
# tft.pca(subdf.iloc[0], 3)

In [4]:
c = df[['close', 'volume', 'diff', 'arb_signal']]
a_df = c.rolling(6).mean().bfill().rename(columns=lambda x: x+'_mean')
b_df = c.rolling(6).std().bfill().rename(columns=lambda x: x+'_std')
c_df = c.rolling(6).skew().bfill().rename(columns=lambda x: x+'_skew')
d_df = c.rolling(6).kurt().bfill().rename(columns=lambda x: x+'_kurt')
df = pd.concat([df, a_df, b_df, c_df, d_df], axis=1).dropna(axis=1)
df.head()

,api,exchange,trading_pair,timestamp,period,open,close,high,low,volume,...,diff_std,arb_signal_std,close_skew,volume_skew,diff_skew,arb_signal_skew,close_kurt,volume_kurt,diff_kurt,arb_signal_kurt
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-06 06:00:00,bitfinex,bitfinex,eth_btc,1546754400,300.0,0.039829,0.039829,0.039829,0.039828,197.424839,...,0.000029,0.028192,-0.269907,0.780103,-0.015923,0.478313,-0.417309,-0.249835,-1.893313,-0.417291
2019-01-06 06:05:00,bitfinex,bitfinex,eth_btc,1546754700,300.0,0.039829,0.039867,0.039892,0.039829,320.041091,...,0.000029,0.028192,-0.269907,0.780103,-0.015923,0.478313,-0.417309,-0.249835,-1.893313,-0.417291
2019-01-06 06:10:00,bitfinex,bitfinex,eth_btc,1546755000,300.0,0.039869,0.039870,0.039870,0.039869,70.079043,...,0.000029,0.028192,-0.269907,0.780103,-0.015923,0.478313,-0.417309,-0.249835,-1.893313,-0.417291
2019-01-06 06:15:00,bitfinex,bitfinex,eth_btc,1546755300,300.0,0.039870,0.039840,0.039870,0.039840,18.358491,...,0.000029,0.028192,-0.269907,0.780103,-0.015923,0.478313,-0.417309,-0.249835,-1.893313,-0.417291
2019-01-06 06:20:00,bitfinex,bitfinex,eth_btc,1546755600,300.0,0.039840,0.039804,0.039841,0.039804,9.974051,...,0.000029,0.028192,-0.269907,0.780103,-0.015923,0.478313,-0.417309,-0.249835,-1.893313,-0.417291


In [5]:
df_sub = df.drop(['timestamp', 'period', 'open', 'high', 'low', 'api', 'exchange', 'trading_pair'], axis=1)
df_sub.head(100).tail(5)

,close,volume,diff,avg,arb_diff,arb_signal,close_mean,volume_mean,diff_mean,arb_signal_mean,...,diff_std,arb_signal_std,close_skew,volume_skew,diff_skew,arb_signal_skew,close_kurt,volume_kurt,diff_kurt,arb_signal_kurt
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-06 13:55:00,0.039729,0.500000,0.000000,0.03973053000000000000,-0.000002,-0.003851,0.039809,232.655774,0.000058,-0.027294,...,0.000052,0.013613,-0.762827,2.448399,1.176434,1.170686,-0.307668,5.995825,1.717661,0.687486
2019-01-06 14:00:00,0.039711,1.875097,0.000005,0.03971950250000000000,-0.000009,-0.021406,0.039783,8.797410,0.000034,-0.026414,...,0.000030,0.013829,-0.341278,1.101697,0.400790,0.853191,-2.067035,0.898237,-0.658446,-0.304342
2019-01-06 14:05:00,0.039675,28.340272,0.000014,0.03968475000000000000,-0.000010,-0.024569,0.039760,11.300374,0.000023,-0.024232,...,0.000021,0.012685,0.155206,0.834246,0.433314,0.532265,-1.776441,-1.398636,-1.008134,0.236626
2019-01-06 14:10:00,0.039678,65.118284,0.000040,0.03968287750000000000,-0.000005,-0.012291,0.039734,18.284786,0.000025,-0.020108,...,0.000022,0.011672,1.035424,1.732067,0.157933,-0.268904,0.519347,2.685041,-1.937920,0.752463
2019-01-06 14:15:00,0.039632,24.047317,0.000075,0.03964124750000000000,-0.000009,-0.023328,0.039699,21.081815,0.000031,-0.017598,...,0.000030,0.007994,0.175218,1.365211,0.463628,1.261752,-0.117872,1.752466,-1.487914,0.639355


In [30]:
# rcParams['figure.figsize'] = 20,80
# df.plot(subplots=True);

In [31]:
def normalize_df(df):
    df = df.copy()
    df = df._get_numeric_data()
    # normalize each point by dividing by the inital point and subtracting 1
    for col in df.columns: 
        df[col] = (df[col] / (df[col][0] + 1e-5)) - 1
    return df
    
def denormalize_results(close_preds):
    return (1 + close_preds) * df.close[0]

In [32]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []
    
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        
        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])
    
    return np.array(data), np.array(labels)

In [54]:
TRAIN_SPLIT = 3000
# dataset = normalize_df(df._get_numeric_data()).values
dataset = normalize_df(df._get_numeric_data()).values
target = df.columns.get_loc('close') - 1
y = dataset[:, target]
past_history = 720
future_target = 5
STEP = 3


x_train, y_train = multivariate_data(dataset, y, 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=False)
x_val, y_val = multivariate_data(dataset, y,
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=False)

In [55]:
len(dataset)

15000

In [70]:
x_train.shape, y_train.shape

((2280, 240, 6), (2280, 5))

In [57]:
print ('Single window of past history : {}'.format(x_train[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train[0].shape))

Single window of past history : (240, 6)

 Target temperature to predict : (5,)


In [58]:
x_train.shape[-2:]

(240, 6)

In [59]:
import tensorflow as tf
BUFFER_SIZE = 10_000
BATCH_SIZE = 256
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()

In [60]:
import tensorflow as tf
import tensorflow.keras.layers as layers

def lstm_model(train):
    model = tf.keras.models.Sequential()
    model.add(layers.Dense(128, use_bias=False))
    model.add(layers.LayerNormalization())
    model.add(layers.PReLU())
    model.add(layers.Dropout(0.20))
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.LSTM(128, activation='relu'))
              
    model.add(layers.Dense(future_target))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='mse')
    return model

U transition Matrix
$h_t = \phi(W x_t + U h_{t-1})$

In [61]:
# single_step_history = single_step_model.fit(train_data, epochs=7,
#                                             steps_per_epoch=38,
#                                             shuffle=False)                                            
##                                             validation_data=(x_val_single, y_val_single))

model = lstm_model(x_train)

history = model.fit(train_data,
                    epochs=9, 
                    steps_per_epoch=38,
                    use_multiprocessing=True,
                    workers=4,
                    validation_data=val_data,
                    validation_steps=5) #(x_val, y_val))

Epoch 1/9
38/38 [==============================] - 27s 700ms/step - loss: 0.0043 - val_loss: 0.0180
Epoch 2/9
38/38 [==============================] - 22s 582ms/step - loss: 0.0011 - val_loss: 0.0108
Epoch 3/9
38/38 [==============================] - 22s 585ms/step - loss: 7.6650e-04 - val_loss: 0.0036
Epoch 4/9
38/38 [==============================] - 22s 581ms/step - loss: 5.9912e-04 - val_loss: 0.0020
Epoch 5/9
38/38 [==============================] - 22s 583ms/step - loss: 5.0218e-04 - val_loss: 0.0011
Epoch 6/9
38/38 [==============================] - 22s 581ms/step - loss: 4.2397e-04 - val_loss: 8.2852e-04
Epoch 7/9
38/38 [==============================] - 22s 580ms/step - loss: 3.8069e-04 - val_loss: 4.3494e-04
Epoch 8/9
38/38 [==============================] - 22s 582ms/step - loss: 3.3156e-04 - val_loss: 4.6111e-04
Epoch 9/9
38/38 [==============================] - 22s 581ms/step - loss: 3.1232e-04 - val_loss: 0.0010


In [62]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              multiple                  768       
_________________________________________________________________
layer_normalization_4 (Layer multiple                  256       
_________________________________________________________________
p_re_lu_4 (PReLU)            multiple                  30720     
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0         
_________________________________________________________________
dense_9 (Dense)              multiple                  16512     
_________________________________________________________________
batch_normalization_2 (Batch multiple                  512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    multiple                 

In [63]:
def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12, 6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)
    
    plt.plot(num_in, np.array(history[:, 1]), label='History')
    plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
        label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
            label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()
    
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(len(loss))
    
    plt.figure()
    
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()
    
    plt.show()


    
plot_train_history(history, 'Multi Step Training and validation loss') 

In [64]:
y

array([-0.00025101,  0.00070283,  0.00077813, ..., -0.12339667,
       -0.12317076, -0.12309546])

In [65]:
preds = denormalize_results(model.predict(x_train))
actual = df.close

# Predictions on the train set

In [66]:
rcParams['figure.figsize'] = 20,3
plt.plot(np.arange(2000), d.denoise(actual[past_history:2000+past_history], 5), label='actual')
plt.plot(range(2000), d.denoise(preds[:, 0][:2000], 5), label='predicted');
plt.legend();

In [67]:
plt.title('All predictions')
plt.plot(range(2000), d.denoise(preds[:2000], 5));

# Predictions on validation set

In [69]:
val_preds = denormalize_results(model.predict(x_val))
# val_actual = df.close.iloc[TRAIN_SPLIT:]
val_actual = denormalize_results(y_val[:, 0])
plt.plot(np.arange(2000), d.denoise(val_actual[past_history:2000+past_history], 20), label='actual')
plt.plot(range(2000), d.denoise(val_preds[:, 0][:2000], 20), label='predicted');
plt.legend();